# Dask Array

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Dask"
* https://docs.dask.org/en/latest/array.html

In [38]:
!pip install graphviz 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива. Визуализируйте граф вычисления задачи.

In [2]:
import numpy as np
import h5py
with h5py.File("demo.h5", "w") as hdf:
    hdf.create_dataset('arr', data=np.random.normal(0, 1, size = (1000, 300_000)))

In [3]:
# with h5py.File("demo.h5", "r") as hdf:
#     # ...
# # ...

hdf = h5py.File("demo.h5", "r")
dset = hdf["arr"]
dset

<HDF5 dataset "arr": shape (1000, 300000), type "<f8">

In [4]:
arr = da.from_array(dset, chunks=(1000, 20000))
arr

dask.array<array, shape=(1000, 300000), dtype=float64, chunksize=(1000, 20000), chunktype=numpy.ndarray>

In [5]:
arr.mean()
# arr.mean().visualize()

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [6]:
(arr.mean() + 5) / 2

dask.array<truediv, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [7]:
arr.mean().compute()

-3.0316597760870505e-05

In [8]:
arr1 = arr / 2
m = arr1.mean()
s = arr1.sum()
m

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [9]:
%%time
s.compute(), m.compute()

Wall time: 18.1 s


(-4547.489664130576, -1.5158298880435252e-05)

In [10]:
%%time
s_c, m_c = dask.compute(
    s,
    m
)
s_c, m_c

Wall time: 6.29 s


(-4547.489664130576, -1.5158298880435252e-05)

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [11]:
%%time
arr_np = np.random.normal(0, 1, size = (1000, 300_000))
(arr_np ** 2).sum()

Wall time: 50.7 s


299983766.64191854

In [12]:
%%time
arr_da = da.random.normal(0, 1, size = (1000, 300_000))
(arr_da ** 2).sum().compute()

Wall time: 8.04 s


300014587.09841657

In [13]:
def f(arr):
    return arr + 2

In [14]:
f(arr_da)

dask.array<add, shape=(1000, 300000), dtype=float64, chunksize=(1000, 15000), chunktype=numpy.ndarray>

In [15]:
def g(arr):
    return np.mean(arr)


In [16]:
g(arr).compute()

-3.0316597760870505e-05

In [17]:
g(np.random.randint(0, 1, (100, 100)))

0.0

In [18]:
%%time
arr1 = arr / 2
# m = arr1.mean().persist()
m = arr1.mean().compute()
m

Wall time: 14.9 s


-1.5158298880435252e-05

In [19]:
%%time
arr1.mean().compute()

Wall time: 12.1 s


-1.5158298880435252e-05

In [20]:
arr[:50, :50].compute()

array([[ 1.15599456,  0.04898961, -1.36550373, ..., -0.13819942,
         0.36295281,  0.64182372],
       [-0.179706  ,  0.02840117,  0.15572787, ...,  0.94572958,
         2.56145198, -0.72746019],
       [ 0.40426528,  0.23525858, -0.05484852, ..., -1.24043863,
        -0.63100602,  0.25350252],
       ...,
       [-1.1216879 , -0.82084612, -0.66371585, ...,  2.06499394,
         0.69888132, -0.04836113],
       [-0.6210468 ,  0.25081832,  0.50124632, ...,  0.06370724,
         2.01715972, -1.6961476 ],
       [ 0.5478802 ,  0.92049107,  2.4512487 , ...,  0.40028398,
        -0.21074185,  0.83947934]])

In [21]:
arr[arr>0].compute()

array([1.15599456, 0.04898961, 1.31883283, ..., 0.25698459, 2.44057592,
       0.50605552])

## Лабораторная работа 11

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений (расчет средних значений, расчет косинусной близости и т.д.) проводятся над `dask.array` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.array` к `numpy.array` или `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.array` (то есть то, что было получено до вызова `compute`, а не после).

1\. Считайте датасет `embeddings` из файла `recipe_embeddings.h5` в виде `dask.array`. Выведите на экран основную информацию о массиве: размер, форму, тип, количество и размер сегментов. 

In [38]:
import dask.array as da
import h5py
import numpy as np
import dask
import pandas as pd

In [40]:
hdf = h5py.File('recipe_embeddings.h5', "r")
dset = hdf['embeddings']

In [41]:
recipe_embeddings = da.from_array(dset)

In [42]:
recipe_embeddings

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(100000, 312), chunktype=numpy.ndarray>

2\. Посчитайте и выведите на экран среднее значение всех элементов массива. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

Пусть $M$ - количество строк в массиве, $N$ - количество столбцов в массиве, `chunks=(r,c)`. Сравните несколько вариантов:
* $r=M$, $с \ll N$ , 
* $r \ll M$, $c=N$ 
* $r = M$, $c = N$ 
* значения $r, c$ по умолчанию.

Выберите наиболее оптимальные значения $r$ и  $c$ в смысле скорости вычислений и далее продолжайте работу с ними.

In [48]:
%%time
recipe_embeddings = da.from_array(dset, chunks = (1200000, 39))
recipe_embeddings.mean().compute()

Wall time: 3.31 s


0.0023777573

In [49]:
%%time
recipe_embeddings = da.from_array(dset, chunks = (12000, 312))
recipe_embeddings.mean().compute()

Wall time: 957 ms


0.0023777566

In [50]:
%%time
recipe_embeddings = da.from_array(dset, chunks = (1200000, 312))
recipe_embeddings.mean().compute()

Wall time: 1.13 s


0.0023777678

In [51]:
%%time
recipe_embeddings = da.from_array(dset)
recipe_embeddings.mean().compute()

Wall time: 878 ms


0.0023777566

In [52]:
#Последний вариант - оптимальный, оставляем recipe_embeddings как в последней ячейке

3\. Опишите пространство, в котором расположены эмбеддинги, посчитав минимальное и максимальное значение для каждой из координат. Сведите результаты в таблицу `pd.DataFrame`, состоящую из двух строк и 312 столбцов. Задайте индексы строк "min" и "max". Названия столбцов сделайте вида $x_i$. Выведите полученную таблицу на экран.

Решите задачу двумя способами. В первом варианте сделайте два вызова метода `compute` для расчета каждого из векторов максимальных и минимальных значений. Во втором варианте сделайте один вызов функции `dask.compute` для одновременного расчета двух векторов. Сравните время выполнения двух решений.

In [53]:
%%time
col_min = recipe_embeddings.min(axis=0).compute()
col_max = recipe_embeddings.max(axis=0).compute()
minmaxdf = pd.DataFrame([col_min, col_max], index=['min', 'max'], columns=[f'x{i}' for i in np.arange(recipe_embeddings.shape[1])] )
minmaxdf

Wall time: 1.72 s


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x302,x303,x304,x305,x306,x307,x308,x309,x310,x311
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


In [54]:
%%time
minn = recipe_embeddings.min(axis=0)
maxx = recipe_embeddings.max(axis=0)
col_min, col_max = dask.compute(
    minn,
    maxx
)
minmaxdf = pd.DataFrame([col_min, col_max], index=['min', 'max'], columns=[f'x{i}' for i in np.arange(recipe_embeddings.shape[1])] )
minmaxdf

Wall time: 899 ms


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x302,x303,x304,x305,x306,x307,x308,x309,x310,x311
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


In [55]:
#При вызове compute один раз для одновременного расчета двух векторов, значения рассчитываются быстрее

4\. Найдите вектор $x \ne x_{256}$ из набора данных, ближайший к вектору $x_{256}$ в смысле метрики $L_1$. Выведите на экран первые 10 координат вектора $x$.

$$d_1(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{|x_k - y_k|}, \textbf{x}, \textbf{y} \in \mathbb{R}^n$$

In [56]:
recipe_embeddings[:10, da.absolute((recipe_embeddings[:, np.delete(np.arange(recipe_embeddings.shape[1]), 256)] - recipe_embeddings[:,256] \
                                    .reshape(-1,1))).sum(axis=0).argmin()].compute()

array([-0.03620593, -0.02302776, -0.02423452, -0.04748494, -0.03022445,
       -0.00947769,  0.01703103, -0.00542553,  0.01890532, -0.00433748],
      dtype=float32)

5\. Рецепты разбиты на 4 группы. Загрузите маску для разбиения на группы из датасета `mask` из файла `recipe_embeddings.h5` в виде `dask.array`. Для каждой группы посчитайте и выведите на экран максимальное значение  нормы $\ell_1$ векторов рецептов, принадлежащих к этой группе. 

Подсказка: закодируйте маску принадлежности к группе при помощи метода кодирования one-hot encoding и воспользуйтесь механизмом распространения.

$$\ell_1: ||\textbf{x}||_1=\sum_{k=1}^{n}{|x_k|}, \textbf{x} \in \mathbb{R}^n$$

In [57]:
hdf = h5py.File('recipe_embeddings.h5', "r")
dset = hdf['mask']

In [58]:
recipe_mask = da.from_array(dset)
recipe_mask

dask.array<array, shape=(1200000,), dtype=int64, chunksize=(1200000,), chunktype=numpy.ndarray>

In [59]:
ohe = (recipe_mask.reshape(-1,1) == da.arange(da.unique(recipe_mask).compute().shape[0]))
ohe

dask.array<eq, shape=(1200000, 4), dtype=bool, chunksize=(1200000, 4), chunktype=numpy.ndarray>

In [60]:
(da.absolute(recipe_embeddings).sum(axis=1).reshape(-1, 1) * ohe).max(axis=0).compute()

array([13.319677, 13.324095, 13.31526 , 13.319157], dtype=float32)

6\. Пусть $X=[\textbf{x}_1,...\textbf{x}_M]^\top$ - матрица эмбеддингов рецептов размера $M\times N$, $W=[\textbf{w}_1,...,\textbf{w}_N]^\top$ - матрица коэффициентов некоторой модели машинного обучения размера $N\times 4$, $y=[y_1,...,y_M]^\top$ - вектор размера $M$, содержащий номера групп рецептов (метки классов). Тогда задачу классификации можно решить следующим образом: $$\hat{y_i} = argmax_j{<X_{i\cdot}, W_{\cdot j}>}$$ где $A_{i\cdot}$ обозначает $i$ строку матрицы, $A_{\cdot j}$ обозначает $j$ столбец матрицы, $\hat{y_i}$ - прогноз класса для рецепта $i$, $<\cdot, \cdot>$ - скалярное произведение векторов.

Инициализируйте матрицу $W$ случайным образом и получите прогнозы для всех рецептов при помощи этой матрицы и матрицы эмбеддингов. Подсчитайте и выведите на экран значение accuracy на основе полученных прогнозов $\hat{y}$ и правильных ответов $y$.

In [79]:
w = da.random.random((recipe_embeddings.shape[1], 4))
w

dask.array<random_sample, shape=(312, 4), dtype=float64, chunksize=(312, 4), chunktype=numpy.ndarray>

In [80]:
ypred = (recipe_embeddings@w).argmax(axis=1)
ypred

dask.array<arg_agg-aggregate, shape=(1200000,), dtype=int64, chunksize=(100000,), chunktype=numpy.ndarray>

In [81]:
accuracy = (ypred == recipe_mask).sum()/recipe_mask.shape[0]
accuracy.compute()

0.216665

7\. Сингулярным разложением (SVD) матрицы $A$ размера $M\times N$ называется разложение вида $A = USV^\top$, где $U$ - матрица размера $M\times N$  ортонормированных векторов произведения $AA^\top$, $V^T$ - транспонированная матрица размера $N\times N$ ортонормированных векторов произведения $A^\top A$, $S$ - диагональная матрица сингулярных значений размера $N\times N$.

SVD может быть использовано для понижения размерности векторов. Для этого от матрицы $U$ оставляют первые $k$ столбцов $U_{\cdot,:k}$, от матрицы $S$ оставляют левый верхний квадрат размера $k\times k$ $S_{:k,:k}$ и вычисляется произведение $\hat{A} = U_{\cdot,:k}S_{:k,:k}$

Выберите эмбеддинги тех рецептов, которые относятся к группе с номеров 3, и уменьшите их размерность до 64 при помощи реализации алгоритма SVD из пакета `dask.array.linalg`. Выведите количество строк и столбцов полученного массива.

Примечание: после отбора рецепта, принадлежащих третьей группе, вызовите у полученного массива метод `compute_chunk_sizes`, чтобы `dask` обновил метаинформацию в этом массиве. 

In [82]:
third_group = recipe_embeddings[ohe[:,3]]
third_group = third_group.compute_chunk_sizes()

In [83]:
u, s, v = da.linalg.svd(third_group)

In [84]:
u = u[:, :64]
s = s[:64]
s = da.diag(s)

In [85]:
Apred = u@s
Apred.shape

(10000, 64)

8\. Используя эмбеддинги уменьшенной размерности, полученные в задании 6, посчитайте косинусное сходство между каждой парой рецептов третьей группы. Выведите матрицу косинусного сходства на экран.

In [86]:
ApredT = Apred.transpose()

In [87]:
cossimilarity = da.dot(Apred, ApredT)/(da.linalg.norm(Apred, axis=1) * da.linalg.norm(Apred, axis=1).reshape(-1, 1))
cossimilarity.compute()

C:\anaconda\lib\site-packages\dask\array\routines.py:264: PerformanceWarning: Increasing number of chunks by factor of 12
  intermediate = blockwise(
C:\anaconda\lib\site-packages\dask\array\core.py:3949: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(


array([[0.99999977, 0.77996181, 0.70962772, ..., 0.78566258, 0.72498306,
        0.70748976],
       [0.77996181, 1.00000006, 0.64223853, ..., 0.86709919, 0.61588698,
        0.65344269],
       [0.70962772, 0.64223853, 1.00000007, ..., 0.64595623, 0.87130879,
        0.8715023 ],
       ...,
       [0.78566258, 0.86709919, 0.64595623, ..., 0.99999999, 0.64296734,
        0.6637292 ],
       [0.72498306, 0.61588698, 0.87130879, ..., 0.64296734, 0.99999983,
        0.91261627],
       [0.70748976, 0.65344269, 0.8715023 , ..., 0.6637292 , 0.91261627,
        1.00000002]])

9\. Посчитайте и выведите на экран количество рецептов, для которых рецепт с индексом `242` входит число топ-5 ближайших рецептов в смысле косинусной близости. При поиске топ-5 рецептов для конкретного рецепта считайте, что он сам в это число не входит.

In [88]:
((cossimilarity.argtopk(6) == 242).sum() -1).compute()

8

10\. Графом называется совокупность двух множеств $G=(V,E)$: множества $V=\{v_1, ..., v_M\}$ узлов и множества ребер $E=\{(v_i, v_j)|v_i\in V, v_j\in V\}$, соединяющих эти узлы. Матрицей смежности невзвешенного графа называется квадратная матрица $A=[a_{ij}]$, в которой ${a_{ij}}$ обозначает количество ребер, соединяющих вершины $i$ и $j$.

Постройте матрицу смежности для графа рецептов на основе матрицы косинусного сходства между каждой парой рецептов. Будем считать, что между двумя рецептами в этом графе существует ребро, если косинусное сходство между двумя этими рецептами не менее 0.85. Петли (ребра из вершины в саму в себя) в графе должны отсутствовать. Посчитайте и выведите на экран количество ребер в данном графе. Проверьте, является ли полученная матрица смежности симметричной.

Примечание: считайте, что два различных рецепта не могут иметь косинусное сходство, равное 1.

In [89]:
smej = cossimilarity * da.logical_not(da.diag(da.ones(cossimilarity.shape[0]))).astype(int)
smej = (smej >= 0.85).astype(int)
smej.compute().sum()

23832772

In [90]:
(smej == smej.T).all().compute()

False

11\. Работая с исходным файлом в формате `h5`, реализуйте алгоритм подсчета среднего вектора датасета в блочной форме.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных и проведение вычислений над этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент. При работе с `h5` вы можете работать с массивами `numpy.array`. Для итерации по сегментам файла допускается использование циклов.

Сравните время и результаты решения работы вашего алгоритма с реализацией поиска среднего вектора из `dask`. 

In [113]:
%%time
recipe_embeddings.mean(axis=0).compute()

Wall time: 748 ms


array([ 9.75613308e-04, -3.39259915e-02,  4.66551557e-02, -8.74476507e-02,
       -9.71177034e-03,  5.51532954e-03,  2.71454044e-02,  3.75370905e-02,
       -2.35138610e-02, -1.10451411e-02,  2.61558332e-02,  8.86423327e-03,
        1.82165187e-02,  5.94256371e-02,  2.72058621e-02, -6.64571533e-03,
        3.04658655e-02,  1.03974920e-02,  1.93780344e-02,  1.43449485e-01,
       -2.46166624e-03, -5.94153255e-03, -3.18063907e-02, -4.10590284e-02,
        8.61882269e-02,  2.80386396e-02, -2.43214611e-02, -4.59721498e-03,
        1.09600713e-02,  3.78382765e-02, -1.06729204e-02, -1.57453716e-02,
        1.08159299e-03, -2.18519215e-02,  7.02830730e-03,  5.17747067e-02,
       -9.01941909e-04, -3.80985737e-02, -7.50487596e-02,  1.78839285e-02,
       -6.01360276e-02,  1.52848035e-01,  7.50525817e-02, -3.78861800e-02,
       -2.17200909e-02,  3.46800871e-03,  3.78258862e-02, -4.40570116e-02,
        4.40716483e-02, -3.56613398e-02,  8.02602526e-03, -3.51645462e-02,
       -4.23886068e-02,  

In [114]:
%%time
hdf = h5py.File('recipe_embeddings.h5', "r")
dset = hdf['embeddings']

mean_array = np.array([], 'float64')
fragments = int(dset.shape[0]/100000)

for f in range(1, fragments+1):
    mean_array = np.append(mean_array, np.array([dset[(f-1)*100000:f*100000].mean(axis=0)]))
       
mean_array.reshape(fragments, -1).mean(axis=0)

Wall time: 1.02 s


array([ 9.75613240e-04, -3.39259934e-02,  4.66551557e-02, -8.74476402e-02,
       -9.71177042e-03,  5.51532962e-03,  2.71454030e-02,  3.75370886e-02,
       -2.35138612e-02, -1.10451417e-02,  2.61558332e-02,  8.86423350e-03,
        1.82165187e-02,  5.94256412e-02,  2.72058623e-02, -6.64571510e-03,
        3.04658658e-02,  1.03974917e-02,  1.93780349e-02,  1.43449499e-01,
       -2.46166628e-03, -5.94153271e-03, -3.18063882e-02, -4.10590296e-02,
        8.61882350e-02,  2.80386388e-02, -2.43214606e-02, -4.59721525e-03,
        1.09600721e-02,  3.78382768e-02, -1.06729195e-02, -1.57453706e-02,
        1.08159300e-03, -2.18519202e-02,  7.02830726e-03,  5.17747070e-02,
       -9.01941904e-04, -3.80985743e-02, -7.50487664e-02,  1.78839285e-02,
       -6.01360320e-02,  1.52848039e-01,  7.50525724e-02, -3.78861800e-02,
       -2.17200910e-02,  3.46800830e-03,  3.78258868e-02, -4.40570122e-02,
        4.40716480e-02, -3.56613376e-02,  8.02602510e-03, -3.51645456e-02,
       -4.23886062e-02,  